In [1]:
import osmium
import geopandas
import folium

class CounterHandler(osmium.SimpleHandler):
    def __init__(self):
        super(CounterHandler, self).__init__()
        self.num_ways = 0
        self.num_tagged_ways = 0
        self.values = set()

    def count(self, tags):
        self.num_ways += 1
        if tags.get('Regelung'):
            self.num_tagged_ways += 1
            self.values.add(tags.get('Regelung'))

    def way(self, w):
        self.count(w.tags)

    # def node(self, n):
    #     self.count(n.tags)

    # def relation(self, r):
    #     self.count(r.tags)

h = CounterHandler()
h.apply_file("data/Schongebiete.shp.osm")
print("Number of ways: %d" % h.num_ways)    
print("Number of tagged ways: %d" % h.num_tagged_ways)    
print(f"Tags: {h.values}")

Number of ways: 234
Number of tagged ways: 225
Tags: {'Bitte den gesamten Winter nicht betreten oder befahren!', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!', 'Skitouren- und Schneeschuhgeher: Bitte vom 1.4. bis 31.5. nicht betreten oder befahren!'}


In [2]:
myshpfile = geopandas.read_file('data/Schongebiete/Schongebiete.shp')
# print(f"Input crs: {myshpfile.crs}")
myshpfile = myshpfile.to_crs("EPSG:4326")
myshpfile.to_file('data/Schongebiete.geojson', driver='GeoJSON')
with open("data/Schongebiete.geojson", "r") as text_file:
    geojson = text_file.read()

In [3]:
def style_function(feature):
    if feature ['properties']['Regelung']:
        return {
            "fillOpacity": 0.5,
            "fillColor": "yellow"
        }
    else:
        return {
            "fillOpacity": 0.5,
            "fillColor": "red"
        }

m = folium.Map(
    location=[47.68, 11.98],
    zoom_start=12,
)
folium.GeoJson(
    geojson, 
    name="geojson", 
    style_function=style_function,
    popup = folium.GeoJsonPopup(
        fields=['Regelung'],
        aliases=['Regelung']
    )    
).add_to(m)
m.save("SchongebieteTags.html")